In [1]:
import numpy as np
import pandas as pd

In [14]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2004.xlsx", encoding='utf8')

In [15]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,16h00,Quarta-feira,21/04/2004,Botafogo,Goiás,1x4,Goiás,1ª Rodada,Caio Martins
1,16h00,Quarta-feira,21/04/2004,Figueirense,Internacional,1x0,Figueirense,1ª Rodada,Orlando Scarpelli
2,16h00,Quarta-feira,21/04/2004,São Caetano,Vitória,1x0,São Caetano,1ª Rodada,Anacleto Campanella
3,18h00,Quarta-feira,21/04/2004,Cruzeiro,Juventude,2x1,Cruzeiro,1ª Rodada,Ipatingão
4,18h00,Quarta-feira,21/04/2004,Palmeiras,Atlético-MG,0x0,-,1ª Rodada,Parque Antártica


** Get all clubes for championship **



In [16]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [17]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [18]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,botafogo
1,figueirense
2,são caetano
3,cruzeiro
4,palmeiras
5,grêmio
6,ponte preta
7,paraná
8,coritiba
9,vasco


In [19]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [23]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    if c1 == u'são caetano':
        pg  -= 24  #Foi punido e perdeu 24 pontos
        
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,botafogo,51.0,46.0,11.0,18.0,17.0,62.0,71.0,-9.0
1,figueirense,63.0,46.0,17.0,12.0,17.0,57.0,59.0,-2.0
2,são caetano,53.0,46.0,23.0,8.0,15.0,65.0,49.0,16.0
3,cruzeiro,56.0,46.0,16.0,8.0,22.0,69.0,81.0,-12.0
4,palmeiras,79.0,46.0,22.0,13.0,11.0,72.0,47.0,25.0
5,grêmio,39.0,46.0,9.0,12.0,25.0,60.0,80.0,-20.0
6,ponte preta,64.0,46.0,19.0,7.0,20.0,43.0,73.0,-30.0
7,paraná,54.0,46.0,15.0,9.0,22.0,52.0,73.0,-21.0
8,coritiba,62.0,46.0,15.0,17.0,14.0,53.0,48.0,5.0
9,vasco,54.0,46.0,14.0,12.0,20.0,64.0,68.0,-4.0


In [24]:
reorderlist = [ 'Santos', 'Atlético-pr', 'São Paulo', 'Palmeiras', 'Corinthians', 'Goiás',
                'Juventude', 'Internacional', 'Fluminense', 'Ponte Preta', 'Figueirense',
                'Coritiba', 'Cruzeiro', 'Paysandu', 'Paraná', 'Vasco', 'Flamengo', 'São Caetano',
                'Atlético-mg', 'Botafogo', 'Criciúma', 'Guarani', 'Vitória', 'Grêmio']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,santos,89.0,46.0,27.0,8.0,11.0,103.0,58.0,45.0
2,atlético-pr,86.0,46.0,25.0,11.0,10.0,93.0,56.0,37.0
3,são paulo,82.0,46.0,24.0,10.0,12.0,78.0,43.0,35.0
4,palmeiras,79.0,46.0,22.0,13.0,11.0,72.0,47.0,25.0
5,corinthians,74.0,46.0,20.0,14.0,12.0,54.0,54.0,0.0
6,goiás,72.0,46.0,21.0,9.0,16.0,81.0,68.0,13.0
7,juventude,70.0,46.0,20.0,10.0,16.0,60.0,66.0,-6.0
8,internacional,67.0,46.0,20.0,7.0,19.0,65.0,58.0,7.0
9,fluminense,67.0,46.0,18.0,13.0,15.0,65.0,68.0,-3.0
10,ponte preta,64.0,46.0,19.0,7.0,20.0,43.0,73.0,-30.0
